### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [3]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
# __________end of block__________

In [4]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cuda device is available


In [5]:
!python -m wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt


Saved under onegin (1).txt


#### 1. Загрузка данных.

In [6]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
    
with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
    
text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
file_len = len(text)
file_len

141888

#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [8]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [47]:
# do not change the code in the block below
# __________start of block__________
batch_size = 1400
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________    

Пример батча:

In [48]:
next(generate_chunk())

array([[83, 45, 47, ..., 53, 57,  1],
       [83, 58, 45, ..., 53, 52, 55],
       [83, 59, 50, ...,  1, 53,  1],
       ...,
       [83, 61, 45, ..., 63, 45,  5],
       [83,  1, 48, ..., 58, 45, 49],
       [83,  1, 46, ...,  1, 58, 50]])

Далее вам предстоит написать код для обучения модели и генерации текста.

В качестве иллюстрации ниже доступен график значений функции потерь, построенный в ходе обучения авторской сети (сам код для ее обучения вам и предстоит написать).

In [49]:

import string
import random
import re
chunk_len = 100

def random_chunk():
    sentences=[]
    for i in range(batch_size):

        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        sentences.append(text[start_index:end_index])
    return sentences

print(random_chunk())

[' залу; дальше: никого.\nдверь отворил он. что ж его\nс такою силой поражает?\nкнягиня перед ним, одна,\nс', 'ль мимо пролетит она,\nвсё благо: бдения и сна\nприходит час определенный;\nблагословен и день забот,\nбл', ' так душно!\nоткрой окно да сядь ко мне». —\n«что, таня, что с тобой?» – «мне скучно,\nпоговорим о стари', 'е своей.\nпод занавескою шелковой\nне спится ей в постеле новой,\nи ранний звон колоколов,\nпредтеча утре', 'ятясь невинным,\nброжу над озером пустынным,\nи far niente мой закон.\nя каждым утром пробужден\nдля слад', 'ртог ко мне златой !..\n\n\n\nxiii\n\nно ленский, не имев, конечно,\nохоты узы брака несть,\nс онегиным желал', 'думой.\nупрямо смотрит он: она\nсидит покойна и вольна.\n\n\n\nxxiii\n\nприходит муж. он прерывает\nсей неприя', 'на тон мужчин,\nна толки про роман туманный,\nна вензель, двум сестрицам данный,\nна ложь журналов, на в', ' обо всем молчал;\nтатьяна изнывала тайно;\nодна бы няня знать могла,\nда недогадлива была.\n\n\n\nxix\n\nвесь', ' на с

In [50]:
class CharDataset:
    def __init__(self, sentences):
        self.data = sentences
   
        self.bos_id = token_to_idx['<sos>']



    def __getitem__(self, idx: int) :
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [token_to_idx.get(char) for char in self.data[idx]]
        

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [51]:
def collate_fn_with_padding(
    input_batch) :
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [52]:
from torch.utils.data import Dataset, DataLoader
train_dataset = CharDataset(random_chunk())
train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=256)
eval_dataset = CharDataset(random_chunk())
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=256)


In [53]:
class CharLM(nn.Module):
    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [54]:
def evaluate(model, criterion) -> float:
    """
    Calculate perplexity on validation dataloader.
    """

    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in eval_dataloader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(
                logits,
                batch['target_ids'].flatten()
                )
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

In [55]:
model = CharLM(hidden_dim=256, vocab_size=num_tokens).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [56]:
from tqdm.auto import tqdm
num_epoch = 10
losses = []
perplexities = []

for epoch in range(num_epoch):
    epoch_losses = []
    model.train()
    for batch in tqdm(train_dataloader, desc=f'Training epoch {epoch}:'):
        optimizer.zero_grad()
        logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
        loss = criterion(
            logits, batch['target_ids'].flatten())
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    losses.append(sum(epoch_losses) / len(epoch_losses))
    perplexities.append(evaluate(model, criterion))

Training epoch 0::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 5::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 6::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 7::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 8::   0%|          | 0/6 [00:00<?, ?it/s]

Training epoch 9::   0%|          | 0/6 [00:00<?, ?it/s]

In [57]:
def generate_sequence(model, starting_seq: str, max_seq_len: int = 500) -> str:
    device = 'cpu'
    model = model.to(device)
    input_ids = [token_to_idx['<sos>']] + [
        token_to_idx.get(char,) for char in starting_seq]
    input_ids = torch.LongTensor(input_ids).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            next_char_distribution = model(input_ids)[-1]
            next_char = next_char_distribution.squeeze().argmax()
            input_ids = torch.cat([input_ids, next_char.unsqueeze(0)])

            

    words = ''.join([idx_to_token[idx.item()] for idx in input_ids])

    return words

In [59]:
generate_sequence(model, starting_seq=' мой дядя самых честных правил')

'<sos> мой дядя самых честных правиль столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь столь '

Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [21]:
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    '''
    
    if seed_phrase is not None:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
    else: 
        x_sequence = [token_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    
    #feed the seed phrase, if any
            
    # your code here
    
    return ''.join([tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]])

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [ ]:
print(generate_sample(model, ' мой дядя самых честных правил ', max_length=500, temperature=0.8))

<sos> мой дядя самых честных правил  а


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [23]:
seed_phrase = ' мой дядя самых честных правил'

In [24]:
generated_phrases = # your code here

# For example:

# generated_phrases = [
#     generate_sample(
#         model,
#         ' мой дядя самых честных правил',
#         max_length=500,
#         temperature=1.
#     ).replace('<sos>', '')
#     for _ in range(10)
# ]

SyntaxError: invalid syntax (695078495.py, line 1)

In [79]:
# do not change the code in the block below
# __________start of block__________

import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'
    

submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')
# __________end of block__________

File saved to `submission_dict.json`


На этом задание завершено. Поздравляем!